<a href="https://colab.research.google.com/github/ehBeak/KkamPpackE/blob/main/traintest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/articulated-animation/articulated-animation-main/articulated-animation-main

/content/drive/MyDrive/articulated-animation/articulated-animation-main/articulated-animation-main


기본 세팅

In [ ]:
!pip install -r requirements.txt

To train a model run:

The code will create a folder in the log directory (each run will create a time-stamped new folder). Checkpoints will be saved to this folder. To check the loss values during training see log.txt. You can also check training data reconstructions in the train-vis subfolder. 

In [ ]:
# dataset_name.yaml 정의 => params 설정하기
!CUDA_VISIBLE_DEVICES=0 python run.py --config config/dataset_name.yaml --device_ids 0

Then to train Animation via disentaglement (AVD) use:

Where {folder} is the name of the folder created in the previous step. (Note: use backslash '' before space.) This will use the same folder where checkpoint was previously stored. It will create a new checkpoint containing all the previous models and the trained avd_network. You can monitor performance in log file and visualizations in train-vis folder.

In [ ]:
# {folder} : 이전 단계에서 만들어진 폴더이름
# https://github.com/AliaksandrSiarohin/pose-evaluation // pose에 관한 인식코드(그외 표정도 있음)
!CUDA_VISIBLE_DEVICES=0 python run.py --checkpoint log/{folder}/cpk.pth --config config/dataset_name.yaml --device_ids 0 --mode train_avd

GPU: 1 V100
dataset: 256x256
trainning time : 2days


[http://jaynewho.com/post/8]

**Checkpoint** : 학습시킨 딥러닝 모델을 저장하는 방법 
=> 학습된 모델의 variable 값 저장

**tf.train.Saver()** : Checkpoint 파일을 저장해주는 클래스

checkpoint
train1.ckpt.data-00000-of-00001
train1.ckpt.index
train1.ckpt.meta

**Checkpoint State Protocol Buffer** 
-> Saver의 saver 모듈을 이용해 모델을 저장할 때, Saver은 Checkpoint State Protocol Buffer를
saved/checkpoint 파일에 담아 저장하고 새로운 job으로 학습할 때마다 업데이트 해 저장한다.

1. model_checkpoint_path : 가장 최근에 저장된 job.ckpt파일의 path정보
2. all_model_checkpoint_paths : 최근에 저장된 job_i.ckpt파일들의 path정보 list
-> 보통 saved 폴더에서 가장 최신의 체크포인트 파일을 불러와 모델을 재학습시키거나 테스트 함
-> 2의 가장 마지막 원소 == 1

**Checkpoint State Protocol Buffer 이용법**
1. tf.train.get_checkpoint_state(saved_dir_path)
2. tf.train.latest_checkpoint(saved_dir_path)

Checkpoint 불러오기


**Trainning**

CUDA_VISIBLE_DEVICES=0 python run.py --config config/dataset_name.yaml --device_ids 0

log dir : Checkpoint 파일들이 저장됨
log.txt : loss value확인가능
train-vis subfolder : trainning data reconstrutions 확인가능

**Animation via disentaglemnt(AVD)**

CUDA_VISIBLE_DEVICES=0 python run.py --checkpoint log/{folder}/cpk.pth --config config/dataset_name.yaml --device_ids 0 --mode train_avd

{folder} : 이전 단계에서 만들어졌던 폴더
이전의 모델들이 들어가있는 체크포인트가 만들어짐

**Evaluation on video reconstruction**
CUDA_VISIBLE_DEVICES=0 python run.py --config config/dataset_name.yaml --mode reconstruction --checkpoint log/{folder}/cpk.pth



In [ ]:
# https://github.com/AliaksandrSiarohin/video-preprocessing
# 사용할 영상 dataset 경로

# requirements -> requirements 다른 경로임
!pip install -r requirements.txt

# Load youtube-dl:  id 본인이 결정
!wget https://yt-dl.org/downloads/latest/youtube-dl -O youtube-dl

!chmod a+rx youtube-dl

#Run script to download videos,
# there are 2 formats that can be used for storing videos one is .mp4 
#and another is folder with .png images. 
#While .png images occupy significantly more space, 
#the format is loss-less and have better i/o performance when training.

!python load_videos.py --metadata taichi-metadata.csv --format .mp4 --out_folder taichi --workers 8


**Training on your own data**

1. Resize all the videos to the same size, e.g 256x256, the videos can be in '.gif', '.mp4' or folder with images. We recommend the latter, for each video make a separate folder with all the frames in '.png' format. This format is loss-less, and it has better i/o performance.

2. Create a folder data/dataset_name with 2 subfolders train and test, put training videos in the train and testing in the test.

3. Create a config file config/dataset_name.yaml. See description of the parameters in the config/vox256.yaml. Specify the dataset root in dataset_params specify by setting root_dir: data/dataset_name. Adjust other parameters as desired, such as the number of epochs for example. Specify id_sampling: False if you do not want to use id_sampling.



1.   비디오를 256x256 사이즈로 만들기(gif, mp4, images 가능)
2.   각각의 비디오는 프레임당 .png포맷으로 만들것임 후자추천
3. 후자가 더 좋은 결과 나온다.
4. data/dataset_name 폴더 만들기
5. data/dataset_name폴더는 2개의 서브폴더를 가짐 각각의 이름은 train과 test임
6. training 비디오는 train폴더에 testing 비디오는 test폴더에 넣기
7. **[config/dataset_yaml이라는 파일 만들기]**
8. dataset_yaml파일에 들어갈 내용은 vox256.yaml을 참고하기
9. dataset root를 지정하기 setting root_dir:data/dataset_name에 dataset_params를 지정하면 된다.
10. 에폭이나 배치사이즈는 알아서 원하는대로 결정하기
11. id_smapling원하지 않으면 id_smpling:False로 지정하기

